In [91]:
import os
import numpy as np
import pandas as pd
from math import ceil
from collections import deque
import math
import sys
global maxdays
current_directory = os.getcwd()
print("Current working directory:", current_directory)

Current working directory: C:\Users\larsv\OneDrive\Documenten\python\COMB


In [92]:
%cd \Users\larsv\OneDrive\Documenten\python\COMB

C:\Users\larsv\OneDrive\Documenten\python\COMB


In [93]:
import pandas as pd
import numpy as np

# Example filepath and filename
directory = "C:\\Users\\larsv\\OneDrive\\Documenten\\python\\COMB"
filename = 'CO_Case2417.txt'

# Read the instance and assign variables to the different values
data = pd.read_csv(filename, sep='\t+', header=None, engine='python')
data = data[0].str.split(expand=True)
title = data.iloc[1, 2] + ' ' + data.iloc[1, 3]
maxdays = int(data.iloc[2, 2])
constraints = data[[0, 2]].iloc[2:5]
constraints = constraints.set_index(0)[2].to_dict()
constraints = dict((k, int(v)) for k, v in constraints.items())
costs = data[[0, 2]].iloc[5:11]
costs = costs.set_index(0)[2].to_dict()
costs = dict((k, int(v)) for k, v in costs.items())
numberMachines = int(data.iloc[11][2])
index = (11 + numberMachines + 1)  # extra step that ensures the right values are taken
machines = np.array(data[[0, 1, 2]][12:index].astype(int))
numberLocations = int(data.iloc[index][2])
locations = np.array(data[[0, 1, 2]][index + 1:(index + numberLocations + 1)].astype(int))
index += numberLocations + 1  # extra step that ensures the right values are taken
depot = locations[0]
numberRequests = int(data.iloc[index][2])
requests = np.array(data[[0, 1, 2, 3, 4, 5]][index + 1:(index + numberRequests + 1)].astype(int))
index += numberRequests + 1  # extra step that ensures the right values are taken
numberTechnicians = int(data.iloc[index][2])
technicians_data = np.array(data[[i for i in range(0, numberMachines + 4)]][index + 1:(index + numberTechnicians + 1)].astype(int))

DAYS = constraints['DAYS']
TRUCK_CAPACITY = constraints['TRUCK_CAPACITY']
TRUCK_MAX_DISTANCE = constraints['TRUCK_MAX_DISTANCE']
TRUCK_DISTANCE_COST = costs['TRUCK_DISTANCE_COST']
TRUCK_DAY_COST = costs['TRUCK_DAY_COST']
TRUCK_COST = costs['TRUCK_COST']
TECHNICIAN_DISTANCE_COST = costs['TECHNICIAN_DISTANCE_COST']
TECHNICIAN_DAY_COST = costs['TECHNICIAN_DAY_COST']
TECHNICIAN_COST = costs['TECHNICIAN_COST']
IDLE_MACHINE_COSTS = {machine[0]: machine[2] for machine in machines}

# Now you can use TECHNICIAN_MAX_DISTANCE in your code

In [94]:
def calculate_distance(coordinates1, coordinates2):
   
    x1, y1 = coordinates1
    x2, y2 = coordinates2
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)


def calculate_distance_between_requests(request_id1, request_id2, requests, locations):
   
    location1 = get_location_details(requests, locations, request_id1)
    location2 = get_location_details(requests, locations, request_id2)
    
    if location1 and location2:
        coordinates1 = (location1[1], location1[2])
        coordinates2 = (location2[1], location2[2])
        distance = calculate_distance(coordinates1, coordinates2)
        return distance
    else:
        return None



def get_location_details(requests, locations, request_id):
    
    for request in requests:
        if request[0] == request_id:  # Check if request ID matches
            location_id = request[1]
            for location in locations:
                if location[0] == location_id:  # Find location details by matching location ID
                    return (location_id, location[1], location[2])  # Return location details
    return None  # Return None if request ID not found

def calculate_request_distance(requests, locations, request_id1, request_id2):
   
    location1 = get_location_details(requests, locations, request_id1)
    location2 = get_location_details(requests, locations, request_id2)
    
    if location1 and location2:
        coordinates1 = (location1[1], location1[2])
        coordinates2 = (location2[1], location2[2])
        print(f"Location corresponding to Request ID {request_id1}: ID={location1[0]}, Coordinates=({location1[1]}, {location1[2]})")
        print(f"Location corresponding to Request ID {request_id2}: ID={location2[0]}, Coordinates=({location2[1]}, {location2[2]})")
        distance = calculate_distance(coordinates1, coordinates2)
        return distance
    else:
        return None

# Example usage:
request_id1_input = 3
request_id2_input = 5

distance = calculate_request_distance(requests, locations, request_id1_input, request_id2_input)
if distance is not None:
    print(f"Distance between Request ID {request_id1_input} and Request ID {request_id2_input}: {distance:.2f}")
else:
    print("One or both request IDs not found.")


def get_truck_routes_with_days(locations, depot, TRUCK_CAPACITY, requests, machines):
    # Sort requests based on the earliest delivery day
    sorted_requests = sorted(requests, key=lambda req: req[2])
    
    depot_loc_ID = depot[0]
    truck_routes = []
    current_truck_route = []
    current_truck_weight = 0
    
    for request_id, location_id, start_day, end_day, machine_kind_id, num_requested_machines in sorted_requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        
        if not current_truck_route:
            current_truck_route.append([request_id])
            current_truck_weight = request_weight
        elif current_truck_weight + request_weight <= TRUCK_CAPACITY:
            # Check if adding the current request exceeds the truck capacity
            # If not, add it to the current truck route
            combined_request_ids = current_truck_route[-1] + [request_id]
            combined_weight = current_truck_weight + request_weight
            if combined_weight <= TRUCK_CAPACITY:
                current_truck_route[-1] = combined_request_ids
                current_truck_weight = combined_weight
            else:
                truck_routes.append(current_truck_route)
                current_truck_route = [[request_id]]
                current_truck_weight = request_weight
        else:
            # If adding the current request exceeds the truck capacity,
            # start a new truck route with the current request
            truck_routes.append(current_truck_route)
            current_truck_route = [[request_id]]
            current_truck_weight = request_weight
    
    # Append the last truck route if it's not empty
    if current_truck_route:
        truck_routes.append(current_truck_route)
    
    return truck_routes


def calculate_delivery_days(truck_routes, requests):
    delivery_days = {}
    for truck_route in truck_routes:
        for location_id, request_ids in enumerate(truck_route, start=1):
            for request_id in request_ids:
                delivery_day = requests[request_id - 1][2]  # Delivery day is at index 2 of each request
                delivery_days[request_id] = delivery_day  # Record the delivery day for each request
    return delivery_days



# Calculate truck routes
truck_routes = get_truck_routes_with_days(locations, depot, TRUCK_CAPACITY, requests, machines)

# Calculate delivery days using truck routes
delivery_days = calculate_delivery_days(truck_routes, requests)




## weet niet of deze nog klopt na aanpassing delivery/installation days
def calculate_idle_days(requests, delivery_days):
    idle_days = {}
    for request_id, installation_day in delivery_days.items():
        delivery_day = requests[request_id - 1][2]  # Delivery day is at index 2 of each request
        machine_kind_id = requests[request_id - 1][4]  # Machine kind ID is at index 4
        idle_days[machine_kind_id] = max(0, installation_day - delivery_day)
    return idle_days

# Calculate idle days
idle_days = calculate_idle_days(requests, delivery_days)


def assign_technicians(delivery_dates, technicians, requests, max_consecutive_days):
    technician_assignments = {}
    consecutive_days = {}
    
    for request_id, installation_day in delivery_dates.items():
        delivery_day = requests[request_id - 1][2]
        earliest_assignment_day = delivery_day + 1
        
        for technician in technicians:
            location_id = technician[1]
            if technician[1] == location_id:
                skill_set = technician[4:]
                if skill_set[requests[request_id - 1][4] - 1] == 1:
                    earliest_available_day = max(earliest_assignment_day, installation_day)
                    
                    if technician[0] not in technician_assignments:
                        technician_assignments[technician[0]] = []
                        consecutive_days[technician[0]] = 0
                    
                    if consecutive_days[technician[0]] < max_consecutive_days:
                        if technician[0] not in consecutive_days:
                            consecutive_days[technician[0]] = 1
                        else:
                            consecutive_days[technician[0]] += 1
                            
                        technician_assignments[technician[0]].append((request_id, earliest_available_day))
                        break
                        
    return technician_assignments

# Example usage with the maximum consecutive days constraint
max_consecutive_days = 5

technician_assignments = assign_technicians(delivery_days, technicians, requests, max_consecutive_days)



# Print technician assignments
for technician_id, assigned_requests in technician_assignments.items():
    technician = [tech for tech in technicians if tech[0] == technician_id][0]
    skill_set = technician[4:]  # Get the skill set of the technician (from index 4 onwards)
    # Get the indices of the skill set where the value is 1
    skill_set_indices = [index + 1 for index, value in enumerate(skill_set) if value == 1]
    print(f"Technician {technician_id} skill set: {skill_set_indices} | Assigned Requests:")
    for request_id, assignment_day in assigned_requests:
        print(f"  - Request {request_id} (Earliest Assignment Day: {assignment_day})")





# Function to calculate penalties for idle days
def calculate_penalties(delivery_days, requests):
    penalties = {}
    for request_id, installation_day in delivery_days.items():
        delivery_day = requests[request_id - 1][2]
        machine_kind_id = requests[request_id - 1][4]
        idle_days = max(0, installation_day - delivery_day - 1)  # Subtract 1 for the day of delivery
        if machine_kind_id not in penalties:
            penalties[machine_kind_id] = 0
        penalties[machine_kind_id] += idle_days
    return penalties


# Function to implement constraints regarding maximum consecutive working days and total distance per day
def apply_constraints(technician_assignments):
    # Implement constraints here
    pass

def get_truck_routes_with_depot(locations, depot, TRUCK_CAPACITY, requests, machines):
    # Sort requests based on the earliest delivery day
    sorted_requests = sorted(requests, key=lambda req: req[2])
    
    depot_loc_ID = depot[0]
    truck_routes = []
    current_truck_route = []
    current_truck_weight = 0
    
    for request_id, location_id, start_day, end_day, machine_kind_id, num_requested_machines in sorted_requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        
        if not current_truck_route:
            current_truck_route.append(depot_loc_ID)  # Start at the depot
            current_truck_route.append(location_id)  # Add the request location
            current_truck_weight = request_weight
        elif current_truck_weight + request_weight <= TRUCK_CAPACITY:
            # Check if adding the current request exceeds the truck capacity
            # If not, add it to the current truck route
            current_truck_route.append(location_id)  # Add the request location
            combined_weight = current_truck_weight + request_weight
            if combined_weight <= TRUCK_CAPACITY:
                current_truck_weight = combined_weight
            else:
                current_truck_route.append(depot_loc_ID)  # End at the depot
                truck_routes.append(current_truck_route)
                current_truck_route = [depot_loc_ID, location_id]  # Start a new truck route
                current_truck_weight = request_weight
        else:
            # If adding the current request exceeds the truck capacity,
            # start a new truck route with the current request
            current_truck_route.append(depot_loc_ID)  # End at the depot
            truck_routes.append(current_truck_route)
            current_truck_route = [depot_loc_ID, location_id]  # Start a new truck route
            current_truck_weight = request_weight
    
    # Append the last truck route if it's not empty
    if current_truck_route:
        current_truck_route.append(depot_loc_ID)  # End at the depot
        truck_routes.append(current_truck_route)
    
    return truck_routes

# Calculate truck routes with depot
truck_routes_with_depot = get_truck_routes_with_depot(locations, depot, TRUCK_CAPACITY, requests, machines)

# Print the truck routes with depot
print("Truck Routes with Depot:")
for idx, truck_route in enumerate(truck_routes_with_depot, start=1):
    print(f"Truck {idx} route: {truck_route}")




def print_truck_routes_with_depot(truck_routes, requests, machines, idle_days, delivery_days, locations):
    total_distance_traveled = 0
    
    for idx, truck_route in enumerate(truck_routes, start=1):
        total_distance_truck = 0
        print(f"Truck {idx} route:")
        total_weight = 0
        
        # Start at depot
        print(f"  Location ID 1 ->", end="")
        
        for request_ids in truck_route:
            for i, request_id in enumerate(request_ids):
                # Print the location ID and request ID
                location_id = requests[request_id - 1][1]
                print(f" Location ID {location_id} ->", end="")
                
                if i < len(request_ids) - 1:
                    print(f" Location ID 1 ->", end="")
                
                if i == len(request_ids) - 1:
                    print(f" Location ID 1")
                    print(f"  Request {request_id}")
                else:
                    print(f" Request {request_id}")
            

# Example usage
print_truck_routes_with_depot(truck_routes, requests, machines, idle_days, delivery_days, locations)

total_distance_traveled = 0

def calculate_distance(locations, route):
    total_distance = 0
    for i in range(len(route) - 1):
        loc1 = locations[route[i] - 1]
        loc2 = locations[route[i + 1] - 1]
        distance = ((loc1[1] - loc2[1]) ** 2 + (loc1[2] - loc2[2]) ** 2) ** 0.5  # Euclidean distance
        total_distance += distance
    return total_distance

def print_truck_routes_with_distances(truck_routes, locations):
    global total_distance_traveled  # Declare as global to modify the variable outside the function
    print("Truck Routes with Distances:")
    for idx, truck_route in enumerate(truck_routes, start=1):
        distance = calculate_distance(locations, truck_route)
        total_distance_traveled += distance
        print(f"Truck {idx} route: {truck_route}, Distance: {distance:.2f}")
    print(f"Total distance traveled by all trucks: {total_distance_traveled:.2f}")

# Example usage
print_truck_routes_with_distances(truck_routes_with_depot, locations)
print(f"Total distance traveled: {total_distance_traveled:.2f}")



def generate_technician_routes_of_requests(technician_assignments):
    technician_routes = {}
    for technician_id, assigned_requests in technician_assignments.items():
        route = [request_id for request_id, _ in assigned_requests]
        technician_routes[technician_id] = route
    return technician_routes

# Generate technician routes
technician_routes = generate_technician_routes_of_requests(technician_assignments)

# Print technician routes
for technician_id, route in technician_routes.items():
    print(f"Technician {technician_id} route in requests:", route)
    

def convert_routes_to_locations(technician_routes, requests):
    technician_routes_locations = {}
    for technician_id, route in technician_routes.items():
        route_locations = []
        for request_id in route:
            location_id = get_location_details(requests, locations, request_id)[0]
            route_locations.append(location_id)
        technician_routes_locations[technician_id] = route_locations
    return technician_routes_locations

# Convert technician routes to locations
technician_routes_locations = convert_routes_to_locations(technician_routes, requests)


def print_technician_routes(technician_routes_locations):
    for technician_id, route in technician_routes_locations.items():
        print(f"Technician {technician_id} route in locations: {route}")
        
# Print technician routes
print_technician_routes(technician_routes_locations)


def calculate_distance_for_tech(loc1, loc2):
    distance = np.linalg.norm(loc2 - loc1)  # Calculate Euclidean distance between coordinates
    return distance

global total_distance_all_technicians

def calculate_technician_distances(technician_routes_locations, locations):
    global total_distance_all_technicians  # Declare the variable as global
    total_distance_all_technicians = 0  # Initialize total distance for all technicians
    technician_distances = {}
    for technician_id, route in technician_routes_locations.items():
        total_distance = 0
        for i in range(len(route) - 1):
            loc1_id = route[i] - 1  # Subtract 1 to adjust for 0-based indexing
            loc2_id = route[i + 1] - 1  # Subtract 1 to adjust for 0-based indexing
            loc1 = locations[loc1_id][1:]  # Extract coordinates from location
            loc2 = locations[loc2_id][1:]  # Extract coordinates from location
            print(f"Loc1: {loc1}, Loc2: {loc2}")
            if isinstance(loc1, np.ndarray) and isinstance(loc2, np.ndarray):  # Check if loc1 and loc2 are numpy arrays
                distance = calculate_distance_for_tech(loc1, loc2)
                total_distance += distance
        technician_distances[technician_id] = total_distance
        total_distance_all_technicians += total_distance  # Add distance for this technician to total distance
    print(f"Total distance for all technicians: {total_distance_all_technicians}")  # Print total distance
    return technician_distances


# Calculate technician distances
technician_distances = calculate_technician_distances(technician_routes_locations, locations)

# Print technician distances
for technician_id, distance in technician_distances.items():
    print(f"Technician {technician_id} distance: {distance}")





technician_assignments = assign_technicians(delivery_days, technicians, requests, max_consecutive_days)
penalties = calculate_penalties(delivery_days, requests)
apply_constraints(technician_assignments)

Location corresponding to Request ID 3: ID=2, Coordinates=(351, 220)
Location corresponding to Request ID 5: ID=4, Coordinates=(367, 427)
Distance between Request ID 3 and Request ID 5: 207.62
Technician 1 skill set: [1, 3] | Assigned Requests:
  - Request 1 (Earliest Assignment Day: 2)
  - Request 2 (Earliest Assignment Day: 2)
  - Request 3 (Earliest Assignment Day: 2)
  - Request 5 (Earliest Assignment Day: 3)
  - Request 6 (Earliest Assignment Day: 4)
Technician 2 skill set: [1, 2, 3] | Assigned Requests:
  - Request 4 (Earliest Assignment Day: 3)
  - Request 7 (Earliest Assignment Day: 4)
  - Request 8 (Earliest Assignment Day: 4)
  - Request 9 (Earliest Assignment Day: 4)
  - Request 10 (Earliest Assignment Day: 4)
Technician 3 skill set: [3] | Assigned Requests:
  - Request 11 (Earliest Assignment Day: 4)
  - Request 14 (Earliest Assignment Day: 4)
  - Request 18 (Earliest Assignment Day: 6)
  - Request 19 (Earliest Assignment Day: 7)
Technician 4 skill set: [1, 2] | Assigned Re

In [86]:
def print_schedule_to_file(truck_routes, technician_assignments, delivery_days, requests, idle_days, machines, output_file):
    with open(output_file, 'w') as f:
        f.write("DATASET = ORTEC Caroline VeRoLog 2019\n")
        f.write("NAME = Instance 1\n")
        f.write("\n")
        
        current_day = 1
        unique_technicians_used = set()
        unique_trucks_used = set()
        unique_days_with_technicians = set()
        unique_days_with_trucks = set()
        
        # Calculate total unique technicians and trucks used
        for tech_id, _ in technician_assignments.items():
            unique_technicians_used.add(tech_id)
        
        for route in truck_routes:
            for request_id in route[0]:
                unique_trucks_used.add(request_id)
        
        # Calculate unique days with technicians and trucks
        for _, assignments in technician_assignments.items():
            for _, day in assignments:
                unique_days_with_technicians.add(day)
        
        for route in truck_routes:
            unique_days_with_trucks.add(delivery_days[route[0][0]])
        
        # Print the total counts of unique technicians and trucks used and number of days
        num_unique_technicians_used = len(unique_technicians_used)
        num_unique_trucks_used = len(unique_trucks_used)
        num_days_with_technicians = len(unique_days_with_technicians)
        num_days_with_trucks = len(unique_days_with_trucks)
        f.write(f"TRUCK_DISTANCE: {int(total_distance_traveled)}\n")
        f.write(f"NUMBER_OF_TRUCK_DAYS = {num_days_with_trucks}\n")
        f.write(f"NUMBER_OF_TRUCKS_USED = {num_unique_trucks_used}\n")
        f.write(f"NUMBER_OF_TECHNICIAN_DAYS = {num_days_with_technicians}\n")
        f.write(f"NUMBER_OF_TECHNICIANS_USED = {num_unique_technicians_used}\n\n")
        
        
        # Print the costs for idle machines
        total_idle_cost = 0
        for machine_kind_id, idle_days_count in idle_days.items():
            machine_cost_per_day = [machine[2] for machine in machines if machine[0] == machine_kind_id][0]
            idle_cost = idle_days_count * machine_cost_per_day
            total_idle_cost += idle_cost
        
        f.write(f"IDLE_MACHINE_COSTS = {total_idle_cost}\n\n")
        
        # Print the schedule for each day
        while True:
            f.write("\n")  # Write an empty line before each day
            f.write(f"DAY = {current_day}\n")
            
            # Print truck routes for the current day
            truck_routes_for_day = [route for route in truck_routes if delivery_days[route[0][0]] == current_day]
            num_trucks_for_day = len(truck_routes_for_day)
            if num_trucks_for_day > 0:
                f.write(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}\n")
                for truck_index, route in enumerate(truck_routes_for_day, start=1):
                    request_ids = " ".join(str(request_id) for request_id in route)
                    f.write(f"{truck_index} {request_ids}\n")
            else:
                f.write("NUMBER_OF_TRUCKS = 0\n")
            
            # Print technician assignments for the current day
            technician_assignments_for_day = [(tech_id, request_id) for tech_id, assignments in technician_assignments.items() for request_id, day in assignments if day == current_day]
            num_technicians_for_day = len(set(tech_id for tech_id, request_id in technician_assignments_for_day))
            if num_technicians_for_day > 0:
                f.write(f"NUMBER_OF_TECHNICIANS = {num_technicians_for_day}\n")
                technician_assignments_dict = {}
                for tech_id, request_id in technician_assignments_for_day:
                    if tech_id not in technician_assignments_dict:
                        technician_assignments_dict[tech_id] = []
                    technician_assignments_dict[tech_id].append(str(request_id))
                for tech_id, assigned_requests in technician_assignments_dict.items():
                    f.write(f"{tech_id} {' '.join(assigned_requests)}\n")
            else:
                f.write("NUMBER_OF_TECHNICIANS = 0\n")
            
            # Check if there are no more routes or assignments for the next day
            if not truck_routes_for_day and not technician_assignments_for_day:
                break
            
            current_day += 1

# Example usage
print_schedule_to_file(truck_routes, technician_assignments, delivery_days, requests, idle_days, machines, 'output.txt')


In [87]:
technician_routes_locations = {
    1: [1, 2, 3, 4],  # Technician 1's route: Locations 1, 2, 3, 4
    2: [1, 5, 6, 4]   # Technician 2's route: Locations 1, 5, 6, 4
}

# Sample data for locations
locations = {
    0: [1, 10, 20],  # Location 1: [x, y] coordinates (x=10, y=20)
    1: [2, 15, 25],  # Location 2: [x, y] coordinates (x=15, y=25)
    2: [3, 20, 30],  # Location 3: [x, y] coordinates (x=20, y=30)
    3: [4, 25, 35],  # Location 4: [x, y] coordinates (x=25, y=35)
    4: [5, 30, 40],  # Location 5: [x, y] coordinates (x=30, y=40)
    5: [6, 35, 45],  # Location 6: [x, y] coordinates (x=35, y=45)
}

def calculate_distance(coordinates1, coordinates2):
    x1, y1 = coordinates1
    x2, y2 = coordinates2
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def calculate_technician_distances(technician_routes_locations, locations, max_distance):
    technician_distances = {}
    total_distance_all_technicians = 0  # Initialize total distance for all technicians
    
    for technician_id, route in technician_routes_locations.items():
        total_distance = 0
        prev_location = None
        for location_id in route:
            if prev_location is not None:
                loc1_id = prev_location - 1  # Subtract 1 to adjust for 0-based indexing
                loc2_id = location_id - 1  # Subtract 1 to adjust for 0-based indexing
                loc1 = np.array(locations[loc1_id][1:])  # Convert to numpy array
                loc2 = np.array(locations[loc2_id][1:])  # Convert to numpy array
                if isinstance(loc1, np.ndarray) and isinstance(loc2, np.ndarray):  # Check if loc1 and loc2 are numpy arrays
                    distance = calculate_distance(loc1, loc2)
                    total_distance += distance
                    if total_distance > max_distance:
                        total_distance -= distance
                        print(f"Technician {technician_id} has exceeded the maximum distance allowed.")
                        break  # Exit the loop if maximum distance is exceeded
            prev_location = location_id
        
        technician_distances[technician_id] = total_distance
        total_distance_all_technicians += total_distance  # Add distance for this technician to total distance
    
    print("Technician distances calculated successfully.")
    print("Total distance for all technicians:", total_distance_all_technicians)
    return technician_distances


# Example usage:
max_distance = 50  # Example maximum distance allowed for technicians
technician_distances = calculate_technician_distances(technician_routes_locations, locations, max_distance)
print("Technician distances:", technician_distances)


Technician distances calculated successfully.
Total distance for all technicians: 70.71067811865476
Technician distances: {1: 21.213203435596427, 2: 49.49747468305833}


In [88]:
def assign_technicians(delivery_dates, technicians, requests, max_consecutive_days, max_distance):
    technician_assignments = {}
    consecutive_days = {}
    
    for request_id, installation_day in delivery_dates.items():
        delivery_day = requests[request_id - 1][2]
        earliest_assignment_day = delivery_day + 1
        
        for technician in technicians:
            technician_id = technician[0]
            location_id = technician[1]
            skill_set = technician[4:]
            
            # Check if the technician has the required skill for the request
            if skill_set[requests[request_id - 1][4] - 1] == 1:
                # Calculate the earliest available day for the technician
                earliest_available_day = max(earliest_assignment_day, installation_day)
                
                # Check if the technician has exceeded the maximum consecutive days
                if technician_id in consecutive_days:
                    if consecutive_days[technician_id] >= max_consecutive_days:
                        continue  # Skip assignment if maximum consecutive days exceeded
                
                # Check if the technician has exceeded the maximum distance for the route
                if technician_id in technician_assignments:
                    current_route_distance = calculate_route_distance(technician_assignments[technician_id], requests)
                    if current_route_distance + calculate_request_distance(request_id, requests) > max_distance:
                        continue  # Skip assignment if maximum distance exceeded
                
                # Assign the request to the technician
                if technician_id not in technician_assignments:
                    technician_assignments[technician_id] = []
                    consecutive_days[technician_id] = 0
                
                # Update consecutive days for the technician
                consecutive_days[technician_id] += 1
                
                technician_assignments[technician_id].append((request_id, earliest_available_day))
                break  # Move to the next request after assignment

    return technician_assignments

def calculate_route_distance(route, requests):
    total_distance = 0
    for request_id, _ in route:
        total_distance += calculate_request_distance(request_id, requests)
    return total_distance

def calculate_request_distance(request_id, requests):
    # Replace this function with your implementation to calculate the distance for a request
    # For example, you can use the coordinates of the locations associated with the request
    return 0  # Placeholder value, replace with actual distance calculation

# Example usage:
max_consecutive_days = 5
max_distance = 100  # Example maximum distance allowed for a route
technician_assignments = assign_technicians(delivery_dates, technicians, requests, max_consecutive_days, max_distance)


NameError: name 'delivery_dates' is not defined